# Table of Content

<a name="outline"></a>

## Setup

- [A](#seca) External Imports
- [B](#secb) Internal Imports
- [C](#secc) Configurations and Paths 
- [D](#secd) JAX Interface
- [E](#sece) General Utility Functions


## Training

- [1](#sec1) Training ICE-NODE and The Baselines


<a name="seca"></a>

### A External Imports [^](#outline)

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'gpu')

<a name="secb"></a>

### B Internal Imports [^](#outline)

In [2]:


sys.path.append("..")

from lib import utils as U
from lib.ehr.dataset import load_dataset

In [3]:
# Assign the folder of the dataset to `DATA_FILE`.

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")

<a name="secd"></a>

### C Configurations and Paths [^](#outline)

In [4]:

output_dir = 'mimic_artefacts'
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [5]:
with U.modified_environ(DATA_DIR=DATA_DIR):
    m3_dataset = load_dataset('M3')
    m4_dataset = load_dataset('M4')
   

                Unrecognised <class 'lib.ehr.coding_scheme.DxICD9'> codes (38)
                to be removed: ['041.49', '282.40', '282.46', '284.11', '284.12', '284.19', '294.20', '294.21', '348.82', '365.70', '425.11', '425.18', '444.09', '512.83', '512.84', '512.89', '516.31', '516.34', '516.36', '518.51', '518.52', '518.53', '573.5', '596.89', '719.70', '747.32', '793.11', '793.19', '795.51', '997.49', '998.01', '998.09', '999.32', '999.33', 'V12.55', 'V13.89', 'V54.82', 'V88.21']
                Unrecognised <class 'lib.ehr.coding_scheme.PrICD9'> codes (7)
                to be removed: ['02.21', '17.55', '17.56', '35.05', '36.01', '36.02', '36.05']
                    Unrecognised <class 'lib.ehr.coding_scheme.DxICD10'> codes (4178)
                    to be removed: ['E10.3213', 'E10.3219', 'E10.3291', 'E10.3292', 'E10.3293', 'E10.3299', 'E10.3312', 'E10.3313', 'E10.3391', 'E10.3393', 'E10.3399', 'E10.3411', 'E10.3413', 'E10.3491', 'E10.3492', 'E10.3511', 'E10.3512', 'E10.3513',

                    Unrecognised <class 'lib.ehr.coding_scheme.DxICD9'> codes (127)
                    to be removed: ['041.49', '173.01', '173.21', '173.22', '173.30', '173.31', '173.32', '173.39', '173.41', '173.42', '173.49', '173.50', '173.51', '173.52', '173.59', '173.60', '173.61', '173.62', '173.69', '173.71', '173.72', '173.79', '173.80', '173.81', '173.82', '173.91', '173.92', '173.99', '282.40', '282.43', '282.44', '282.46', '284.11', '284.12', '284.19', '286.52', '286.59', '294.20', '294.21', '310.81', '310.89', '331.6', '348.82', '358.30', '365.70', '365.71', '365.72', '365.73', '365.74', '379.27', '414.4', '415.13', '425.11', '425.18', '444.09', '488.81', '488.82', '488.89', '508.2', '512.2', '512.81', '512.82', '512.83', '512.84', '512.89', '516.31', '516.32', '516.33', '516.34', '516.36', '516.37', '516.5', '518.51', '518.52', '518.53', '539.01', '539.09', '539.89', '573.5', '596.81', '596.82', '596.83', '596.89', '629.31', '629.32', '631.0', '631.8', '649.81', '704.41'

In [6]:
from lib.ml import ICENODE, ICENODE_UNIFORM, GRU, RETAIN, WindowLogReg
%load_ext autoreload
%autoreload 2
"""
predefined hyperparams re: each model.
"""

model_cls = {
    'ICE-NODE': ICENODE,
    'ICE-NODE_UNIFORM': ICENODE_UNIFORM,
    'GRU': GRU,
    'RETAIN': RETAIN,
    'LogReg': WindowLogReg
}

model_config = {
    'ICE-NODE': f'{SOURCE_DIR}/expt_configs/icenode.json' ,
    'ICE-NODE_UNIFORM': f'{SOURCE_DIR}/expt_configs/icenode.json' ,
    'GRU': f'{SOURCE_DIR}/expt_configs/gru.json' ,
    'RETAIN': f'{SOURCE_DIR}/expt_configs/retain.json',
    'LogReg': f'{SOURCE_DIR}/expt_configs/window_logreg.json'
}

model_config = {clf: U.load_config(file) for clf, file in model_config.items()}

clfs = [
    'ICE-NODE', 
    'ICE-NODE_UNIFORM', 
    'GRU', 'RETAIN', 'LogReg']


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# icenode_variants = []

# timescale = {'Ts7': 7.0, 'Ts30': 30.0}
# embeddings_size = {'E200': 200, 'E250': 250, 'E300': 300}
# tayreg = {'Ty0': 0, 'Ty3': 3}
# for ts_key, ts_val in timescale.items():
#     for e_key, e_val in embeddings_size.items():
#         for tay_key, tay_val in tayreg.items():
#             model_name = f'ICE-NODE-{"".join((ts_key, e_key, tay_key))}'
#             icenode_variants.append(model_name)
            
#             config = U.load_config(f'{SOURCE_DIR}/expt_configs/icenode.json')
#             config['emb']['dx']['embeddings_size'] = e_val
#             config['model']['timescale'] = ts_val
#             config['training']['tay_reg'] = tay_val
#             model_config[model_name] = config
#             model_cls[model_name] = ICENODE
# clfs.extend(icenode_variants)

In [8]:
m3_train_output_dir = {clf: f'{output_dir}/m3_train/{clf}' for clf in clfs}
m4_train_output_dir = {clf: f'{output_dir}/m4_train/{clf}' for clf in clfs}

[Path(d).mkdir(parents=True, exist_ok=True) for d in m3_train_output_dir.values()]
[Path(d).mkdir(parents=True, exist_ok=True) for d in m4_train_output_dir.values()]

[None, None, None, None, None]

In [9]:
from lib.ml import ConfigDiskWriter, MinibatchLogger, EvaluationDiskWriter, ParamsDiskWriter

The reporter objects are called inside training iterations
Each has its own functionality:
1. **ConfigDiskWriter**: writes the experiment config file as JSON in the same training directory
2. **MinibatchLogger**: writes to the console the training progress details.
3. **EvaluationDiskWriter**: writes the evaluation as csv tables in the same training directory for each step of the 100.
4. **ParamsDiskWriter**: writes the model parameters snapshot at each step out of 100.

In [10]:
make_reporters = lambda output_dir, config: [ConfigDiskWriter(output_dir=output_dir, config=config),
                                             MinibatchLogger(config),
                                             EvaluationDiskWriter(output_dir=output_dir),
                                             ParamsDiskWriter(output_dir=output_dir)]

m3_reporters = {model: make_reporters(m3_train_output_dir[model], model_config[model]) for model in clfs}
m4_reporters = {model: make_reporters(m4_train_output_dir[model], model_config[model]) for model in clfs}

<a name="sece"></a>

### D JAX Interface [^](#outline)

In [11]:
from lib.ehr.coding_scheme import DxCCS, DxFlatCCS, DxICD9, DxICD10
from lib.ehr import Subject_JAX
from lib.ehr import StaticInfoFlags

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The dictionary `code_scheme` in the next cell specifies the code spaces of:
- 'dx': diagnostic input (input features) codes. Possible arguments:  `DxCCS()`, `DxFlatCCS()`, `DxICD9()`, `DxICD10()`.
- 'outcome': diagnostic outcome (target to be predicted) codes. Possible arguments: 
    - `OutcomeExtractor('<outcome_label>')`  for prediction of all `<outcome_label>` codes or 
    - `FirstOccurrenceOutcomeExtractor('<outcome_label>')` for prediction of `<outcome_label>` for the first occurrence per patient).
    - `'<outcome_label>'` specifies a subset of dx-codes defined by JSON files in `lib/ehr/resources/outcome_filters`, where each of the labels below links to the following JSON file:
        - `'dx_flatccs_mlhc_groups'`:  `'dx_flatccs_mlhc_groups.json'`,
        - `'dx_flatccs_filter_v1'`: `'dx_flatccs_v1.json'`,
        - `'dx_icd9_filter_v1'`: `'dx_icd9_v1.json'`,
        - `'dx_icd9_filter_v2_groups'`: `'dx_icd9_v2_groups.json'`,
        - `'dx_icd9_filter_v3_groups'`: `'dx_icd9_v3_groups.json'`
    

**Note**: OutcomeExtractor can be configured through a JSON file to focus only a subset of the diagnostic codes. For example,
you can focus the prediction objective on a small subset of interest (e.g. to predict only pulmonary-heart 
diseases codes, etc.).
OutcomeExtractor can also be replaced by FirstOccurrenceOutcomeExtractor to enforce the prediction 
objective to predict only the first occurrence of each code for one patient, and subsequent
redundant occurences will be avoided and not incorporated in the loss function.

In [12]:
from lib.ehr import OutcomeExtractor, SurvivalOutcomeExtractor
outcome_class = OutcomeExtractor

In [13]:
code_scheme = {
    'dx': DxCCS(), # other options 
    'outcome': OutcomeExtractor('dx_flatccs_filter_v1')
}

### Adding Demographic Information in Training

What do you need to include as control features? **Uncomment each line to consider the corresponding static information.**

In [14]:

static_info_flags = StaticInfoFlags(gender=True, age=True)

m3_interface = Subject_JAX.from_dataset(m3_dataset, 
                                        code_scheme=code_scheme, 
                                        static_info_flags=static_info_flags,
                                       data_max_size_gb=1)
m4_interface = Subject_JAX.from_dataset(m4_dataset, 
                                        code_scheme=code_scheme, 
                                        static_info_flags=static_info_flags,
                                       data_max_size_gb=1)

m3_splits = m3_interface.random_splits(split1=0.7, split2=0.85, random_seed=42)
m4_splits = m4_interface.random_splits(split1=0.7, split2=0.85, random_seed=42)


                            S - M_domain (2497, p=0.14371223021582732):
                            ['001', '002', '003', '003.2', '004']...

                            M_domain - S (0, p=0.0):
                            []...

                            M_domain (14878):
                            ['001.0', '001.1', '001.9', '002.0', '002.1']...

                            S (17375): ['001', '001.0', '001.1', '001.9', '002']...

                            S - M_domain (2497, p=0.14371223021582732):
                            ['001', '002', '003', '003.2', '004']...

                            M_domain - S (0, p=0.0):
                            []...

                            M_domain (14878):
                            ['001.0', '001.1', '001.9', '002.0', '002.1']...

                            S (17375): ['001', '001.0', '001.1', '001.9', '002']...
                            dx_icd10->dx_icd9 Unrecognised t_codes
                            (169):
                    

In [15]:
import jax.random as jrandom
import lib.ml as ml
%load_ext autoreload
%autoreload 2
key = jrandom.PRNGKey(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In the next cell we load a dictionary for each model specifiying the experiment configuration per model.
The classname of the trainer used is also specified in the experiment configs.
For example, this is the configuration file of ICE-NODE experiment.

```json
{
    "emb": {
        "dx": {
           "decoder_n_layers": 2,
           "classname":  "MatrixEmbeddings",      
           "embeddings_size": 300
        }
    },
    "model": {
        "ode_dyn_label": "mlp3",
        "ode_init_var": 1e-7,
        "state_size": 30,
        "timescale": 30
    },
    "training": {
        "batch_size": 256,
        "decay_rate": [0.25, 0.33],
        "lr": [7e-5,  1e-3],
        "epochs": 60,
        "reg_hyperparams": {
            "L_dyn": 1000.0,
            "L_l1": 0,
            "L_l2": 0
        },
        "opt": "adam",
        "classname": "ODETrainer2LR" <---- "classname, so this class should be available through ml package."
    }
}
```

Since we have a string of the classname, one way to get `ml.ODETrainer2LR` is `getattr(ml, 'ODETrainer2LR')`

In [16]:
m3_models = {clf: model_cls[clf].from_config(model_config[clf],
                                              m3_interface,
                                              m3_splits[0],
                                              key) for clf in clfs}
m4_models = {clf: model_cls[clf].from_config(model_config[clf],
                                              m4_interface,
                                              m4_splits[0],
                                              key) for clf in clfs}



trainers_cls = {clf: getattr(ml, model_config[clf]["training"]["classname"]) for clf in clfs}
trainers = {clf: trainers_cls[clf](**model_config[clf]["training"]) for clf in clfs}

## Metrics of Interest Specification

In [17]:
from lib.metric import (CodeAUC, UntilFirstCodeAUC, AdmissionAUC, CodeGroupTopAlarmAccuracy, LossMetric, MetricsCollection)

## Evaluation Metrics per Model

1. *CodeAUC*: evaluates the prediction AUC per code (aggregating over all subject visits, for all subjects)
2. *UntilFirstCodeAUC*: same as *CodeAUC*, but evaluates the prediction AUC until the first occurrence for each subject, once the code occured, all the subsequent visits are ignored for that code. If the code does not show in a particular subject, all the subject visits are ignored.
3. *AdmissionAUC*: evaluates the prediction AUC per visit (i.e. probability of assigning higher risk values for present codes than the absent ones).
4. *CodeGroupTopAlarmAccuracy*: partition codes into groups according the code frequency (from the most frequent to the least), and for each visit picks the top `k` risks, and the metric evaluates the accuracy of the top `k` riskiest codes by the model for being indeed present.
5. *LossMetric*: records the loss values for different loss variants, which doesn't necessarily include the actual loss function that was used in the training.
6. *MetricsCollection*: Groups multiple metrics to be considered at once.

**Note:** you will get different results when calling the method `outcome_by_percentiles` by changing the 'outcome' enty of the `ode_scheme` dictionary as following:
- OutcomeExtractor: the counting will consider the code and its redundant occurrence for each subject, then aggregated over all subjects 
- FirstOccurrenceOutcomeExtractor: the counting will consider the first occurrence only for each subject, then aggregated over all subjects.

In [18]:
# pecentile_range=20 will partition the codes into five gruops, where each group contains 
# codes that overall constitutes 20% of the codes in all visits of specified 'subjects' list.
m3_code_freq_partitions = m3_interface.outcome_by_percentiles(percentile_range=20, subjects=m3_splits[0])
m4_code_freq_partitions = m4_interface.outcome_by_percentiles(percentile_range=20, subjects=m4_splits[0])

# Evaluate for different k values
top_k_list = [3, 5, 10, 15, 20]

m3_metrics = [CodeAUC(m3_interface),
              UntilFirstCodeAUC(m3_interface),
              AdmissionAUC(m3_interface),
              LossMetric(m3_interface),
              CodeGroupTopAlarmAccuracy(m3_interface, top_k_list=top_k_list, code_groups=m3_code_freq_partitions)]
m4_metrics = [CodeAUC(m4_interface),
              UntilFirstCodeAUC(m4_interface),
              AdmissionAUC(m4_interface),
              LossMetric(m4_interface),
              CodeGroupTopAlarmAccuracy(m4_interface, top_k_list=top_k_list, code_groups=m4_code_freq_partitions)]

m3_metrics = MetricsCollection(m3_metrics)
m4_metrics = MetricsCollection(m4_metrics)

<a name="sec1"></a>

### 1 Training ICE-NODE and The Baselines (#outline)

In [19]:
from lib.ml import MetricsHistory

def m3_train(clf):
    output_dir = m3_train_output_dir[clf]
    config = model_config[clf]
    model = m3_models[clf]
    trainer = trainers[clf]
    reporters = [EvaluationDiskWriter(output_dir), # <- responsible for writing evaluation tables on disk at the given path
                 ParamsDiskWriter(output_dir), # <- responsible for writing model parameters snapshot after each iteration.
                 ConfigDiskWriter(output_dir, config), # writes the config file as JSON
                ]
    
    history = MetricsHistory(m3_metrics) # <- empty history
    
    return trainer(model, m3_interface, m3_splits, history=history, reporters=reporters, prng_seed=42)

def m4_train(clf):
    output_dir = m4_train_output_dir[clf]
    config = model_config[clf]
    model = m4_models[clf]
    trainer = trainers[clf]
    reporters = [EvaluationDiskWriter(output_dir), # <- responsible for writing evaluation tables on disk at the given path
                 ParamsDiskWriter(output_dir), # <- responsible for writing model parameters snapshot after each iteration.
                 ConfigDiskWriter(output_dir, config), # writes the config file as JSON
                ]
    
    history = MetricsHistory(m4_metrics) # <- empty history
    
    return trainer(model, m4_interface, m4_splits, history=history, reporters=reporters, prng_seed=42)

#### ICE-NODE

In [20]:
# for icenode_variant in icenode_variants:
#     print(icenode_variant)
#     m3_train(icenode_variant)

In [21]:
m3_icenode_results = m3_train('ICE-NODE')

  0%|                                                                                                                                | 0/1439 [00:00<?, ?it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0000.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
  1%|█▏                                                                                                                   | 14/1439 [02:14<2:56:17,  7.42s/it]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0001.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
  2%|██▎                                                                                                                  | 29/1439 [04:17<2:34:50,  6.59s/it]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0002.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
  3%|███▍                   

 25%|█████████████████████████████▎                                                                                      | 363/1439 [48:04<2:06:32,  7.06s/it]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0025.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 26%|██████████████████████████████▍                                                                                     | 377/1439 [49:57<2:07:20,  7.19s/it]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0026.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 27%|███████████████████████████████▌                                                                                    | 392/1439 [51:53<2:00:20,  6.90s/it]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0027.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 28%|███████████████████████

 50%|█████████████████████████████████████████████████████████▌                                                        | 726/1439 [1:35:20<1:20:33,  6.78s/it]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0050.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 51%|██████████████████████████████████████████████████████████▌                                                       | 740/1439 [1:37:11<1:18:05,  6.70s/it]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0051.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 52%|███████████████████████████████████████████████████████████▊                                                      | 755/1439 [1:39:09<1:19:18,  6.96s/it]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0052.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 53%|███████████████████████

 76%|███████████████████████████████████████████████████████████████████████████████████████                            | 1089/1439 [2:23:18<39:44,  6.81s/it]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0075.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 77%|████████████████████████████████████████████████████████████████████████████████████████▏                          | 1103/1439 [2:25:09<38:53,  6.94s/it]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0076.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 78%|█████████████████████████████████████████████████████████████████████████████████████████▎                         | 1118/1439 [2:27:09<37:35,  7.03s/it]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0077.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 79%|███████████████████████

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1439/1439 [3:09:45<00:00,  7.91s/it]


In [ ]:
m4_icenode_results = m4_train('ICE-NODE')

 26%|████████████████████████████▋                                                                                   | 1914/7465 [8:56:04<18:52:26, 12.24s/it]

#### ICE-NODE_UNIFORM

In [22]:
m3_icenode_uniform_results = m3_train('ICE-NODE_UNIFORM')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1439/1439 [2:29:28<00:00,  6.23s/it]


In [ ]:
m4_icenode_uniform_results = m4_train('ICE-NODE_UNIFORM')

#### GRU

In [23]:
m3_gru_results = m3_train('GRU')

  0%|                                                                                                                               | 0/10832 [00:00<?, ?it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0000.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
  1%|█▏                                                                                                                 | 109/10832 [02:15<2:21:33,  1.26it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0001.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
  2%|██▎                                                                                                                | 218/10832 [04:02<2:28:09,  1.19it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0002.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
  3%|███▍                   

 25%|████████████████████████████▊                                                                                     | 2735/10832 [42:18<1:49:14,  1.24it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0025.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 26%|█████████████████████████████▉                                                                                    | 2844/10832 [43:58<1:40:53,  1.32it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0026.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 27%|███████████████████████████████                                                                                   | 2953/10832 [45:36<1:48:59,  1.20it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0027.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 28%|███████████████████████

 50%|████████████████████████████████████████████████████████▌                                                       | 5470/10832 [1:23:53<1:22:30,  1.08it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0050.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 52%|█████████████████████████████████████████████████████████▋                                                      | 5579/10832 [1:25:34<1:08:39,  1.28it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0051.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 53%|██████████████████████████████████████████████████████████▊                                                     | 5689/10832 [1:27:14<1:14:50,  1.15it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0052.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 54%|███████████████████████

 76%|██████████████████████████████████████████████████████████████████████████████████████▎                           | 8205/10832 [2:05:35<35:55,  1.22it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0075.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 77%|███████████████████████████████████████████████████████████████████████████████████████▍                          | 8314/10832 [2:07:15<32:32,  1.29it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0076.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 78%|████████████████████████████████████████████████████████████████████████████████████████▋                         | 8424/10832 [2:08:53<30:00,  1.34it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'step0077.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 79%|███████████████████████

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10832/10832 [2:45:40<00:00,  1.09it/s]


In [ ]:
m4_gru_results = m4_train('GRU')

#### RETAIN

In [ ]:
m3_retain_results = m3_train('RETAIN')

 87%|████████████████████████████████████████████████████████████████████████████████████████████████▉               | 5689/6576 [10:16:23<1:54:40,  7.76s/it]

In [ ]:
m4_retain_results = m4_train('RETAIN')

In [ ]:
m3_logreg_results = m3_train('LogReg')

In [ ]:
m4_logreg_results = m4_train('LogReg')